In [1]:
%matplotlib inline

import io, os, sys, types, time

from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

import pandas as pd

import matplotlib
import matplotlib.pyplot as plt

import numpy as np
from numpy.linalg import eig, inv

import math

from scipy import interpolate, spatial, stats

import seaborn as sns

import skimage.io as skiIo
from skimage import exposure, img_as_float

from sklearn import linear_model
from sklearn import metrics

import cv2

In [2]:
import pickle

In [3]:
""" ============== path settings =============="""

' ============== path settings =============='

In [16]:
global isMac
isMac = True

if(isMac):
#     dataPrefix = '/Users/lily/Lily/Academic/AW_Lab/Projects/NSP/Code/python/Quantification_fromPositionOnly/Trial/Materials_trial'
#     figOutPrefix = '/Users/lily/Lily/Academic/AW_Lab/Projects/NSP/Code/python/Quantification_fromPositionOnly/Trial/FigureOutputs'
#     dataOutPrefix = '/Users/lily/Lily/Academic/AW_Lab/Projects/NSP/Code/python/Quantification_fromPositionOnly/Trial/DataOutputs'
#     dataPrefix = '/Volumes/wji/lab/Projects/NSP/Data_Analysis/Experiments/Fate_Switching/GFP_RFP/Summaries/ver2018/Data'
#     imgDataPrefix = '/Users/lily/Lily/Academic/AW_Lab/data/figureData'
#     dataPrefix = '/Users/lily/Lily/Academic/AW_Lab/Projects/NSP/Data_Analysis/Experiments/Fate_Switching/Data'
#     figOutPrefix = '/Users/lily/Lily/Academic/AW_Lab/Projects/NSP/Data_Analysis/Experiments/Fate_Switching/python_Outputs/Figure'
#     dataOutPrefix = '/Users/lily/Lily/Academic/AW_Lab/Projects/NSP/Data_Analysis/Experiments/Fate_Switching/python_Outputs/Data'
    dataPrefix = '/Volumes/wji/lab/Projects/NSP/Data_Analysis/Experiments/Fate_Switching/GFP_RFP/Summaries/ver2018/Data'
    figOutPrefix = '/Volumes/wji/lab/Projects/NSP/Data_Analysis/Experiments/Fate_Switching/GFP_RFP/Summaries/ver2018/DataOutputs'
    dataOutPrefix = '/Volumes/wji/lab/Projects/NSP/Data_Analysis/Experiments/Fate_Switching/GFP_RFP/Summaries/ver2018/FigureOutputs'

else:
    dataPrefix = 'Z:\\lab\\Projects\\NSP\\Data_Analysis\\Experiments\\Fate_Switching\\GFP_RFP\\Summaries\\ver2018\\Data'
    figOutPrefix = 'Z:\\lab\\Projects\\NSP\\Data_Analysis\\Experiments\\Fate_Switching\\GFP_RFP\\Summaries\\ver2018\\FigureOutputs'
    dataOutPrefix = 'Z:\\lab\\Projects\\NSP\\Data_Analysis\\Experiments\\Fate_Switching\\GFP_RFP\\Summaries\\ver2018\\DataOutputs'



In [17]:
""" ============== Golbal variables and parameters =============="""

' ============== Golbal variables and parameters =============='

In [18]:
### global variables
global ColorCode, targetIndexMatch, bins, channel_mapping
ColorCode = {1:'#00FFFF', 2:'#1FF509', 3: '#FF0000', 4: '#CFCF1C', 5: '#FF00FF', 6: '#FFAE01', 7:'#FF0000', 0:'#FFFFFF'}
targetIndexMatch = {0:0, 1:2, 2:3, 3:4, 4:5, 5:7}
bins = 4096
channel_mapping = {'RFP':0, 'GFP':1, 'R3':2, 'R4':3, 'Cy5':4, 0:'RFP', 1:'GFP', 2:'R3', 3:'R4', 4:'Cy5'}

In [19]:
### analysis parameters
radiusExpanseRatio = 1.5
num_angleSection = 20
num_outsideAngle = 10
num_Xsection = 20
z_offset = 25
analysisParams = [num_angleSection, num_outsideAngle, num_Xsection, z_offset, radiusExpanseRatio]

In [20]:
"""============== importing settings =============="""

'============== importing settings =============='

In [21]:
def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path
    
    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path

class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path
    
    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)
        
        print ("importing Jupyter notebook from %s" % path)
                                       
        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)
        
        
        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod
        
        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__
        
        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod


class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}
    
    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return
        
        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)
        
        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]


sys.meta_path.append(NotebookFinder())

In [22]:
import Data_quantification_functions_v0522 as my

In [23]:
"""============== main =============="""

'============== main =============='

In [24]:
### file parameters
# file folders
imageFolder = 'Images'
ROIFolder = 'ROIs'
summaryFolder = 'Summaries'

# file name
# summaryNames = {}
# summaryNames['Control'] = ('Control_s1r1_summary.csv',)
# summaryNames['Fz'] = ('Fz_s1r1_summary.csv', 'Fz_s4r1_summary.csv')

In [25]:
sumNames = ['Ni_s1r1_summary.csv']
Categories = ['N']
dataNames = ['N_sample1_region1_good_0809.pickle']

In [26]:
len(sumNames)

1

In [27]:
"""Load data"""
Images = {}
ROI_dfs = {}
annot_dfs = {}
image_shapes = {}
M2P_ratios = {}
bundles_dfs = {}
annot_bundles_dfs = {}
annot_bundles_dfs_good = {}
for i in range(len(sumNames)):
    print(i)
    sumName = sumNames[i]
    summary_df = pd.read_csv(os.path.join(dataPrefix, summaryFolder, sumName))
    image_list = summary_df.loc[:,'Image_Name'].unique()
    ROI_list = summary_df.loc[:,'ROI_Name'].unique()
    imageName = image_list[0]
    ROIName = ROI_list[0]
#     %time Images[i] = img_as_float(skiIo.imread(os.path.join(imgDataPrefix, imageName)))
    ROI_dfs[i] = pd.read_csv(os.path.join(dataPrefix, ROIFolder, ROIName))
    ROI_dfs[i].rename(columns = {' ':'No'}, inplace = True)
    annot_dfs[i] = summary_df.groupby(['Image_Name']).get_group(image_list[0]).reset_index(drop = True)
#     image_shapes[i] = (Images[i].shape[0], Images[i].shape[1], Images[i].shape[2])
    M2P_ratios[i] = (summary_df.iloc[0]['imgX_pixel']/summary_df.iloc[0]['imgX_um'], summary_df.iloc[0]['imgY_pixel']/summary_df.iloc[0]['imgY_um'])
    bundles_dfs[i] = my.getBundlesInfo(ROI_dfs[i], annot_dfs[i], M2P_ratios[i][0], M2P_ratios[i][1])
    annot_bundles_dfs[i] = bundles_dfs[i].dropna(axis=0, how='any', inplace = False)
    annot_bundles_dfs_good[i] = my.Good_QC_df(annot_bundles_dfs[i])

0


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
""" Load IntensityMatrix Data """
outputDatas = {}
for i in range(len(dataNames)):
    print(i)
    inputname = os.path.join(dataOutPrefix, Categories[i], dataNames[i])
    pickle_in = open(inputname,"rb")
    outputDatas[i] = pickle.load(pickle_in)


In [ ]:
list(set(Categories))

In [ ]:
""" Parameters """
relAxisParams = outputDatas[0]['Parameter'][0][1][0]
xTicks_ori = np.round(np.linspace(0, radiusExpanseRatio, num_Xsection+1), 2)
yTicks_ori = relAxisParams[5]*-1

In [ ]:
relAxisParams[5]

In [ ]:
yTicks_ori

In [ ]:
tickTypeX = 1
tickTypeY = 2
tickArg2_X = 5
tickArg2_Y = [-2, -1,0, 1, 2]
tickParams = xTicks_ori, yTicks_ori, tickTypeX, tickTypeY, tickArg2_X, tickArg2_Y 

In [ ]:
""" Get R3 and R4(gfp) intensity matrix and boundary conditions """
## initialization
sum_gfpMatrix = {}
sum_r3Matrix = {}
boundaries = {}
for cat in list(set(Categories)):
    sum_gfpMatrix[cat] = {}
    sum_r3Matrix[cat] = {}
    
for i in range(len(dataNames)):
    cat = Categories[i]
    gfp = outputDatas[i]['IntensityMatrix'][0,1,2,:,:,:,:]
    r3 = outputDatas[i]['IntensityMatrix'][0,2,2,:,:,:,:]
    sum_gfp = np.sum(gfp, axis = 3)
    sum_r3 = np.sum(r3, axis = 3)
    sum_gfpMatrix[cat][i] = sum_gfp
    sum_r3Matrix[cat][i] = sum_r3

In [ ]:
def plotSummaryMatrix(matrix, cmap, vmax, tickParams, labelParams):
    sns.set(font_scale=2)
    xTick_ori, yTick_ori, tickTypeX, tickTypeY, tickArg2_X, tickArg2_Y = tickParams
    ylabel, xlabel = labelParams
    xTicks = my.getTickList(tickTypeX, xTick_ori, tickArg2_X)
    yTicks = my.getTickList(tickTypeY, yTick_ori, tickArg2_Y)
    
    ax = plt.subplot(111)
    sns.heatmap(matrix, cmap = cmap, yticklabels = yTicks, xticklabels = xTicks, vmin = 0, vmax = vmax)
    ax.set_ylabel(ylabel)
    ax.set_xlabel(xlabel)
#     ax2.set_title(title, fontsize=16)
#     ax2.invert_yaxis()

In [ ]:
""" Plot sample images """
gfp_smp = outputDatas[0]['IntensityMatrix'][0,1,2,9,:,:,:]
r3_smp = outputDatas[0]['IntensityMatrix'][0,2,2,9,:,:,:]

vmax_gfp = np.ceil(np.percentile(sum_gfpMatrix['Control'][0].flatten(), 99))
vmax_r3 = np.ceil(np.percentile(sum_r3Matrix['Control'][0].flatten(), 90))

ylabel = 'Rel. Angle'
xlabel = 'Rel. Length'
labelParams = ylabel, xlabel

plotSummaryMatrix(np.sum(gfp_smp, axis = 2), 'Greens', vmax_gfp, tickParams, labelParams)


In [ ]:
plotSummaryMatrix(np.sum(r3_smp, axis = 2), 'Reds', vmax_r3, tickParams, labelParams)


In [ ]:
"""Control_summary"""
sum_gfpMatrix['Control']['Sum'] = np.concatenate((sum_gfpMatrix['Control'][0], sum_gfpMatrix['Control'][1]), axis=0)
sum_r3Matrix['Control']['Sum'] = np.concatenate((sum_r3Matrix['Control'][0], sum_r3Matrix['Control'][1]), axis=0)

In [ ]:
sum_gfpMatrix['Control']['Sum'].shape

In [ ]:
vmax_gfp = np.ceil(np.percentile(sum_gfpMatrix['Control']['Sum'].flatten(), 95))
vmax_r3 = np.ceil(np.percentile(sum_r3Matrix['Control']['Sum'].flatten(), 95))

ylabel = 'Rel. Angle'
xlabel = 'Rel. Length'
labelParams = ylabel, xlabel

plotSummaryMatrix(np.mean(sum_r3Matrix['Control']['Sum'], axis = 0), 'Reds', vmax_r3, tickParams, labelParams)

In [ ]:
plotSummaryMatrix(np.mean(sum_gfpMatrix['Control']['Sum'], axis = 0), 'Greens', vmax_gfp, tickParams, labelParams)

In [ ]:
"""Fz perturbed"""

In [ ]:
def getIndexList(bundles_df):
    doubleR3List = []
    doubleR4List = []
    wtList = []
    
    for ind in range(bundles_df.shape[0]):    
        if((bundles_df.iloc[ind]['FL_Green'] == 0) & (bundles_df.iloc[ind]['FL_R3'] == 2)):
#             print('two R3: ', str(ind))
            doubleR3List.append(ind)
        elif((bundles_df.iloc[ind]['FL_Green'] == 1) & (bundles_df.iloc[ind]['FL_R3'] == 1)):
#             print('WT: ', str(ind))
            wtList.append(ind)
        elif((bundles_df.iloc[ind]['FL_Green'] == 2) & (bundles_df.iloc[ind]['FL_R3'] == 0)):
#             print('two R4: ', str(ind))
            doubleR4List.append(ind)
#     print(doubleR3List)
    return (doubleR3List, doubleR4List, wtList)

In [ ]:
annot_bundles_dfs_good[2].index[0], annot_bundles_dfs_good[2].index[3]

In [ ]:
ls_1 = getIndexList(annot_bundles_dfs_good[2])
ls_2 = getIndexList(annot_bundles_dfs_good[3])

In [ ]:
ls_1

In [ ]:
annot_bundles_dfs_good[2].index

In [ ]:
8, 16, 35

In [ ]:
ls_2

In [ ]:
sum_gfpMatrix['Fz']['dR3'] = {}
sum_gfpMatrix['Fz']['dR4'] = {}
sum_gfpMatrix['Fz']['wt'] = {}
sum_r3Matrix['Fz']['dR3'] = {}
sum_r3Matrix['Fz']['dR4'] = {}
sum_r3Matrix['Fz']['wt'] = {}
for ind in [2,3]:
    indList = getIndexList(annot_bundles_dfs_good[ind])
    print(indList)
    sum_gfpMatrix['Fz']['dR3'][ind] = sum_gfpMatrix['Fz'][ind][indList[0], :, :]
    sum_gfpMatrix['Fz']['dR4'][ind] = sum_gfpMatrix['Fz'][ind][indList[1], :, :]
    sum_gfpMatrix['Fz']['wt'][ind] = sum_gfpMatrix['Fz'][ind][indList[2], :, :]
    sum_r3Matrix['Fz']['dR3'][ind] = sum_r3Matrix['Fz'][ind][indList[0], :, :]
    sum_r3Matrix['Fz']['dR4'][ind] = sum_r3Matrix['Fz'][ind][indList[1], :, :]
    sum_r3Matrix['Fz']['wt'][ind] = sum_r3Matrix['Fz'][ind][indList[2], :, :]
sum_gfpMatrix['Fz']['dR3']['Sum'] = np.concatenate((sum_gfpMatrix['Fz']['dR3'][2], sum_gfpMatrix['Fz']['dR3'][3]), axis=0)
sum_gfpMatrix['Fz']['dR4']['Sum'] = np.concatenate((sum_gfpMatrix['Fz']['dR4'][2], sum_gfpMatrix['Fz']['dR4'][3]), axis=0)
sum_gfpMatrix['Fz']['wt']['Sum'] = np.concatenate((sum_gfpMatrix['Fz']['wt'][2], sum_gfpMatrix['Fz']['wt'][3]), axis=0)
sum_r3Matrix['Fz']['dR3']['Sum'] = np.concatenate((sum_r3Matrix['Fz']['dR3'][2], sum_r3Matrix['Fz']['dR3'][3]), axis=0)
sum_r3Matrix['Fz']['dR4']['Sum'] = np.concatenate((sum_r3Matrix['Fz']['dR4'][2], sum_r3Matrix['Fz']['dR4'][3]), axis=0)
sum_r3Matrix['Fz']['wt']['Sum'] = np.concatenate((sum_r3Matrix['Fz']['wt'][2], sum_r3Matrix['Fz']['wt'][3]), axis=0)

In [ ]:
""" Plot sample images """
ind = 17
print(annot_bundles_dfs_good[3].index[ind])
gfp_smp = outputDatas[3]['IntensityMatrix'][0,1,2,ind,:,:,:]
r3_smp = outputDatas[3]['IntensityMatrix'][0,2,2,ind,:,:,:]

vmax_gfp = np.ceil(np.percentile(sum_gfpMatrix['Fz'][2].flatten(), 90))
vmax_r3 = np.ceil(np.percentile(sum_r3Matrix['Fz'][2].flatten(), 90))

ylabel = 'Rel. Angle'
xlabel = 'Rel. Length'
labelParams = ylabel, xlabel

plotSummaryMatrix(np.sum(gfp_smp, axis = 2), 'Greens', 10, tickParams, labelParams)


In [ ]:
plotSummaryMatrix(np.sum(r3_smp, axis = 2), 'Reds', 8, tickParams, labelParams)

In [ ]:
vmax_gfp = np.ceil(np.percentile(sum_gfpMatrix['Fz']['wt']['Sum'].flatten(), 95))
vmax_r3 = np.ceil(np.percentile(sum_r3Matrix['Fz']['wt']['Sum'].flatten(), 95))

ylabel = 'Rel. Angle'
xlabel = 'Rel. Length'
labelParams = ylabel, xlabel

plotSummaryMatrix(np.mean(sum_r3Matrix['Fz']['dR3']['Sum'], axis = 0), 'Reds', vmax_r3, tickParams, labelParams)

In [ ]:
plotSummaryMatrix(np.mean(sum_gfpMatrix['Fz']['dR3']['Sum'], axis = 0), 'Greens', vmax_gfp, tickParams, labelParams)

In [ ]:
plotSummaryMatrix(np.mean(sum_r3Matrix['Fz']['wt']['Sum'], axis = 0), 'Reds', vmax_r3, tickParams, labelParams)

In [ ]:
plotSummaryMatrix(np.mean(sum_gfpMatrix['Fz']['wt']['Sum'], axis = 0), 'Greens', vmax_gfp, tickParams, labelParams)

In [ ]:
vmax_gfp = np.ceil(np.percentile(sum_gfp_ctrl.flatten(), 99))
vmax_r3 = np.ceil(np.percentile(sum_r3_ctrl.flatten(), 90))

In [ ]:
gfp_ctrl = outputDatas[0]['IntensityMatrix'][0,1,2,:,:,:,:]
r3_ctrl = outputDatas[0]['IntensityMatrix'][0,2,2,:,:,:,:]

In [ ]:
gfp_ctrl_2 = outputDatas[3]['IntensityMatrix'][0,1,2,:,:,:,:]
r3_ctrl_2 = outputDatas[3]['IntensityMatrix'][0,2,2,:,:,:,:]

In [ ]:
gfp_ctrl_smp10 = gfp_ctrl[9,:,:,:]
r3_ctrl_smp10 = r3_ctrl[9,:,:,:]

In [ ]:
"""Get min/max params"""
sliceType = 0
percentile = 95
sum_gfp_ctrl_1 = np.sum(gfp_ctrl, axis = 3)
sum_r3_ctrl_1 = np.sum(r3_ctrl, axis = 3)
vmax_gfp_ctrl_1 = np.ceil(np.percentile(sum_gfp_ctrl.flatten(), 99))
vmax_r3_ctrl_1 = np.ceil(np.percentile(sum_r3_ctrl.flatten(), 90))

In [ ]:
sum_gfp_ctrl_2 = np.sum(gfp_ctrl_2, axis = 3)
sum_r3_ctrl_2 = np.sum(r3_ctrl_2, axis = 3)
vmax_gfp_ctrl_2 = np.ceil(np.percentile(sum_gfp_ctrl_2.flatten(), 99))
vmax_r3_ctrl_2 = np.ceil(np.percentile(sum_r3_ctrl_2.flatten(), 90))

In [ ]:
# # tickParams = [1, 2, 5, [-1.5, -1, -0.5, 0, 0.5, 1, 1.5]] ### tickTypeX, tickTypeY, tickArg2_X, tickArg2_Y
# tickTypeX = 1
# tickTypeY = 2
# tickArg2_X = 5
# tickArg2_Y = [-2, -1.5, -1, -0.5, 0, 0.5, 1, 1.5, 2]

# pp = outputDatas[0]['Parameter'][0][0][1]
# xTick_ori = np.round(np.linspace(0, radiusExpanseRatio, gfp_ctrl_smp10[:,:,0].shape[1]), 2)
# xTicks = my.getTickList(tickTypeX, xTick_ori, tickArg2_X)
# yTick_ori = np.round(outputDatas[0]['Parameter'][0][1][0][5], 2)
# yTicks = my.getTickList(tickTypeY, yTick_ori, tickArg2_Y)

# ax2 =  plt.subplot(111)
# #     sns.heatmap(np.sum(intensityMatrix, axis = 2), cmap = colormap, yticklabels = yTicks, xticklabels = xTicks)
# # sns.heatmap(np.mean(sum_GFP_matrix, axis = 2), cmap = 'Greens', yticklabels = False, xticklabels = False, vmin = vim, vmax = vmax)
# sns.heatmap(np.sum(gfp_ctrl_smp10, axis = 2), cmap = 'Greens', yticklabels = yTicks, xticklabels = xTicks, vmin = 0, vmax = vmax_gfp_ctrl)
# # sns.heatmap(gfp_ctrl_smp10[21:22,:,20], cmap = 'Greens')
# ax2.set_ylabel('Rel. Angle', fontsize=16)
# ax2.set_xlabel('Rel. Length', fontsize=16)
# ax2.set_title('Sum over z-stack', fontsize=16)
# ax2.invert_yaxis()




In [ ]:
# ax2 =  plt.subplot(111)
# #     sns.heatmap(np.sum(intensityMatrix, axis = 2), cmap = colormap, yticklabels = yTicks, xticklabels = xTicks)
# # sns.heatmap(np.mean(sum_R3_matrix, axis = 2), cmap = 'Reds', yticklabels = False, xticklabels = False, vmin = vim, vmax = vmax)
# sns.heatmap(np.sum(r3_ctrl_smp10, axis = 2), cmap = 'Reds', yticklabels = yTicks, xticklabels = xTicks, vmin = 0, vmax = vmax_r3_ctrl)


# ax2.set_ylabel('Rel. Angle', fontsize=16)
# ax2.set_xlabel('Rel. Length', fontsize=16)
# ax2.set_title('Sum over z-stack', fontsize=16)
# ax2.invert_yaxis()

In [ ]:
vmax_gfp_ctrl, vmax_r3_ctrl

In [ ]:
""" Control """

In [ ]:
# tickParams = [1, 2, 5, [-1.5, -1, -0.5, 0, 0.5, 1, 1.5]] ### tickTypeX, tickTypeY, tickArg2_X, tickArg2_Y
tickTypeX = 1
tickTypeY = 2
tickArg2_X = 5
tickArg2_Y = [-2, -1.5, -1, -0.5, 0, 0.5, 1, 1.5, 2]

# pp = outputDatas[0]['Parameter'][0][0][1]
xTick_ori = np.round(np.linspace(0, radiusExpanseRatio, sum_gfp_ctrl_1[0,:,:].shape[1]), 2)
xTicks = my.getTickList(tickTypeX, xTick_ori, tickArg2_X)
yTick_ori = np.round(outputDatas[0]['Parameter'][0][1][0][5], 2)
yTicks = my.getTickList(tickTypeY, yTick_ori, tickArg2_Y)

ax2 =  plt.subplot(111)
#     sns.heatmap(np.sum(intensityMatrix, axis = 2), cmap = colormap, yticklabels = yTicks, xticklabels = xTicks)
# sns.heatmap(np.mean(sum_GFP_matrix, axis = 2), cmap = 'Greens', yticklabels = False, xticklabels = False, vmin = vim, vmax = vmax)
sns.heatmap(np.mean(sum_gfp_ctrl_1, axis = 0), cmap = 'Greens', yticklabels = yTicks, xticklabels = xTicks, vmin = 0, vmax = vmax_gfp_ctrl_1)
ax2.set_ylabel('Angle', fontsize=16)
ax2.set_xlabel('length from R-heel', fontsize=16)
ax2.set_title('Sum over z-stack', fontsize=16)
ax2.invert_yaxis()



In [ ]:
ax2 =  plt.subplot(111)
#     sns.heatmap(np.sum(intensityMatrix, axis = 2), cmap = colormap, yticklabels = yTicks, xticklabels = xTicks)
# sns.heatmap(np.mean(sum_R3_matrix, axis = 2), cmap = 'Reds', yticklabels = False, xticklabels = False, vmin = vim, vmax = vmax)
sns.heatmap(np.mean(sum_r3_ctrl_1, axis = 0), cmap = 'Reds', yticklabels = yTicks, xticklabels = xTicks, vmin = 0, vmax = vmax_r3_ctrl_1)
ax2.set_ylabel('Angle', fontsize=16)
ax2.set_xlabel('length from R-heel', fontsize=16)
ax2.set_title('Sum over z-stack', fontsize=16)
ax2.invert_yaxis()

In [ ]:
"Control-2"

In [ ]:
# tickParams = [1, 2, 5, [-1.5, -1, -0.5, 0, 0.5, 1, 1.5]] ### tickTypeX, tickTypeY, tickArg2_X, tickArg2_Y
tickTypeX = 1
tickTypeY = 2
tickArg2_X = 5
tickArg2_Y = [-2, -1.5, -1, -0.5, 0, 0.5, 1, 1.5, 2]

# pp = outputDatas[0]['Parameter'][0][0][1]
xTick_ori = np.round(np.linspace(0, radiusExpanseRatio, sum_gfp_ctrl[0,:,:].shape[1]), 2)
xTicks = my.getTickList(tickTypeX, xTick_ori, tickArg2_X)
# yTick_ori = np.round(pp[5], 2)
yTicks = my.getTickList(tickTypeY, yTick_ori, tickArg2_Y)

ax2 =  plt.subplot(111)
#     sns.heatmap(np.sum(intensityMatrix, axis = 2), cmap = colormap, yticklabels = yTicks, xticklabels = xTicks)
# sns.heatmap(np.mean(sum_GFP_matrix, axis = 2), cmap = 'Greens', yticklabels = False, xticklabels = False, vmin = vim, vmax = vmax)
sns.heatmap(np.mean(sum_gfp_ctrl_2, axis = 0), cmap = 'Greens', yticklabels = yTicks, xticklabels = xTicks, vmin = 0, vmax = vmax_gfp_ctrl_2)
ax2.set_ylabel('Angle', fontsize=16)
ax2.set_xlabel('length from R-heel', fontsize=16)
ax2.set_title('Sum over z-stack', fontsize=16)
ax2.invert_yaxis()

In [ ]:
ax2 =  plt.subplot(111)
#     sns.heatmap(np.sum(intensityMatrix, axis = 2), cmap = colormap, yticklabels = yTicks, xticklabels = xTicks)
# sns.heatmap(np.mean(sum_R3_matrix, axis = 2), cmap = 'Reds', yticklabels = False, xticklabels = False, vmin = vim, vmax = vmax)
sns.heatmap(np.mean(sum_r3_ctrl_2, axis = 0), cmap = 'Reds', yticklabels = yTicks, xticklabels = xTicks, vmin = 0, vmax = vmax_r3_ctrl_2)
ax2.set_ylabel('Angle', fontsize=16)
ax2.set_xlabel('length from R-heel', fontsize=16)
ax2.set_title('Sum over z-stack', fontsize=16)
ax2.invert_yaxis()

In [ ]:
"Sum of controls"

In [ ]:
sum_r3_ctrl_2.shape

In [ ]:
sum_r3_ctrl_1.shape

In [ ]:
sum_gfp_ctrl = np.concatenate((sum_gfp_ctrl_2, sum_gfp_ctrl_1), axis=0, out=None)
sum_r3_ctrl = np.concatenate((sum_r3_ctrl_2, sum_r3_ctrl_1), axis=0, out=None)

In [ ]:
vmax_gfp_ctrl = np.ceil(np.percentile(sum_gfp_ctrl.flatten(), 95))
vmax_r3_ctrl = np.ceil(np.percentile(sum_r3_ctrl.flatten(), 90))

In [ ]:
ax2 =  plt.subplot(111)
#     sns.heatmap(np.sum(intensityMatrix, axis = 2), cmap = colormap, yticklabels = yTicks, xticklabels = xTicks)
# sns.heatmap(np.mean(sum_GFP_matrix, axis = 2), cmap = 'Greens', yticklabels = False, xticklabels = False, vmin = vim, vmax = vmax)
sns.heatmap(np.mean(sum_gfp_ctrl, axis = 0), cmap = 'Greens', yticklabels = yTicks, xticklabels = xTicks, vmin = 0, vmax = vmax_gfp_ctrl)
ax2.set_ylabel('Angle', fontsize=16)
ax2.set_xlabel('length from R-heel', fontsize=16)
ax2.set_title('Sum over z-stack', fontsize=16)
ax2.invert_yaxis()

In [ ]:
ax2 =  plt.subplot(111)
#     sns.heatmap(np.sum(intensityMatrix, axis = 2), cmap = colormap, yticklabels = yTicks, xticklabels = xTicks)
# sns.heatmap(np.mean(sum_R3_matrix, axis = 2), cmap = 'Reds', yticklabels = False, xticklabels = False, vmin = vim, vmax = vmax)
sns.heatmap(np.mean(sum_r3_ctrl, axis = 0), cmap = 'Reds', yticklabels = yTicks, xticklabels = xTicks, vmin = 0, vmax = vmax_r3_ctrl)
ax2.set_ylabel('Angle', fontsize=16)
ax2.set_xlabel('length from R-heel', fontsize=16)
ax2.set_title('Sum over z-stack', fontsize=16)
ax2.invert_yaxis()

In [ ]:
"Fz-1"

In [ ]:
annot_bundles_dfs_good[1].index

In [ ]:
gfp_fz1 = outputDatas[1]['IntensityMatrix'][0,1,2,:,:,:,:]
r3_fz1_R3 = outputDatas[1]['IntensityMatrix'][0,2,0,:,:,:,:]
r3_fz1_R4 = outputDatas[1]['IntensityMatrix'][0,2,1,:,:,:,:]
r3_fz1_T0 = outputDatas[1]['IntensityMatrix'][0,2,2,:,:,:,:]

In [ ]:
a = list(annot_bundles_dfs_good[2].index)
a.index(34)

In [ ]:
annot_bundles_dfs_good[2].index

In [ ]:
gfp_fz2 = outputDatas[2]['IntensityMatrix'][0,1,2,:,:,:,:]
gfp_fz2 = outputDatas[2]['IntensityMatrix'][0,2,2,:,:,:,:]

In [ ]:
fz2_type1_list = [a.index(34), a.index(40), a.index(48), a.index(98)]#34, 40, 48, 98
fz2_type2_list = [a.index(18), a.index(38), a.index(41), a.index(44), a.index(45), a.index(52)]#18, 38, 41, 44, 45, 52

In [ ]:
fz1_type1_list = [1, 4,8,11] #6,8,27,34
fz1_type2_list = [3, 5, 12]#13, 16, 35

In [ ]:
ind = 4;
gfp_fz = gfp_fz1[ind,:,:,:]
r3_fz = r3_fz1_T0[ind,:,:,:]

In [ ]:
gfp_fz1_type1 = gfp_fz1[fz1_type1_list,:,:,:]
gfp_fz1_type2 = gfp_fz1[fz1_type2_list,:,:,:]

r3_fz1_R4_type1 = r3_fz1_R4[fz1_type1_list,:,:,:]
r3_fz1_R4_type2 = r3_fz1_R4[fz1_type2_list,:,:,:]

r3_fz1_R3_type1 = r3_fz1_R3[fz1_type1_list,:,:,:]
r3_fz1_R3_type2 = r3_fz1_R3[fz1_type2_list,:,:,:]

r3_fz1_T0_type1 = r3_fz1_T0[fz1_type1_list,:,:,:]
r3_fz1_T0_type2 = r3_fz1_T0[fz1_type2_list,:,:,:]

In [ ]:
"""Get min/max params"""
sliceType = 0
sum_GFP_matrix = np.sum(gfp_fz1_type1, axis = 3)
sum_r3_R3_matrix = np.sum(r3_fz1_R3_type1, axis = 3)
sum_r3_R4_matrix = np.sum(r3_fz1_R4_type1, axis = 3)
sum_r3_T0_matrix = np.sum(r3_fz1_T0_type1, axis = 3)

vmax_GFP = np.ceil(np.percentile(sum_GFP_matrix.flatten(), 99))
vmax_R3 = np.ceil(np.percentile(sum_r3_R3_matrix.flatten(), 90))



vmax_GFP, vmax_R3



In [ ]:
# tickParams = [1, 2, 5, [-1.5, -1, -0.5, 0, 0.5, 1, 1.5]] ### tickTypeX, tickTypeY, tickArg2_X, tickArg2_Y
tickTypeX = 1
tickTypeY = 2
tickArg2_X = 5
tickArg2_Y = [-2, -1.5, -1, -0.5, 0, 0.5, 1, 1.5, 2]

pp = outputDatas[0]['Parameter'][0][0][1]
xTick_ori = np.round(np.linspace(0, radiusExpanseRatio, sum_GFP_matrix[0,:,:].shape[1]), 2)
xTicks = my.getTickList(tickTypeX, xTick_ori, tickArg2_X)
yTick_ori = np.round(outputDatas[0]['Parameter'][0][1][0][5], 2)
yTicks = my.getTickList(tickTypeY, yTick_ori, tickArg2_Y)

ax2 =  plt.subplot(111)
#     sns.heatmap(np.sum(intensityMatrix, axis = 2), cmap = colormap, yticklabels = yTicks, xticklabels = xTicks)
# sns.heatmap(np.mean(sum_GFP_matrix, axis = 2), cmap = 'Greens', yticklabels = False, xticklabels = False, vmin = vim, vmax = vmax)
sns.heatmap(np.sum(gfp_fz, axis = 2), cmap = 'Greens', yticklabels = yTicks, xticklabels = xTicks, vmin = 0, vmax = vmax_GFP)
ax2.set_ylabel('Angle', fontsize=16)
ax2.set_xlabel('length from R-heel', fontsize=16)
ax2.set_title('Sum over z-stack', fontsize=16)
ax2.invert_yaxis()

In [ ]:
ax2 =  plt.subplot(111)
#     sns.heatmap(np.sum(intensityMatrix, axis = 2), cmap = colormap, yticklabels = yTicks, xticklabels = xTicks)
# sns.heatmap(np.mean(sum_r3_T0_matrix, axis = 0), cmap = 'Reds', yticklabels = yTicks, xticklabels = xTicks, vmin = 0, vmax = vmax_R3)
sns.heatmap(np.sum(r3_fz, axis = 2), cmap = 'Reds', yticklabels = yTicks, xticklabels = xTicks, vmin = 0, vmax = vmax_R3)
ax2.set_ylabel('Angle', fontsize=16)
ax2.set_xlabel('length from R-heel', fontsize=16)
ax2.set_title('Sum over z-stack', fontsize=16)
ax2.invert_yaxis()

In [ ]:
ax2 =  plt.subplot(111)
#     sns.heatmap(np.sum(intensityMatrix, axis = 2), cmap = colormap, yticklabels = yTicks, xticklabels = xTicks)
# sns.heatmap(np.mean(sum_R3_matrix, axis = 2), cmap = 'Reds', yticklabels = False, xticklabels = False, vmin = vim, vmax = vmax)
sns.heatmap(np.mean(sum_r3_R3_matrix, axis = 0), cmap = 'Reds', yticklabels = yTicks, xticklabels = xTicks, vmin = 0, vmax = vmax_R3)
ax2.set_ylabel('Angle', fontsize=16)
ax2.set_xlabel('length from R-heel', fontsize=16)
ax2.set_title('Sum over z-stack', fontsize=16)
ax2.invert_yaxis()

In [ ]:
ax2 =  plt.subplot(111)
#     sns.heatmap(np.sum(intensityMatrix, axis = 2), cmap = colormap, yticklabels = yTicks, xticklabels = xTicks)
# sns.heatmap(np.mean(sum_R3_matrix, axis = 2), cmap = 'Reds', yticklabels = False, xticklabels = False, vmin = vim, vmax = vmax)
sns.heatmap(np.mean(sum_r3_R4_matrix, axis = 0), cmap = 'Reds', yticklabels = yTicks, xticklabels = xTicks, vmin = 0, vmax = vmax_R3)
ax2.set_ylabel('Angle', fontsize=16)
ax2.set_xlabel('length from R-heel', fontsize=16)
ax2.set_title('Sum over z-stack', fontsize=16)
ax2.invert_yaxis()

In [ ]:
ax2 =  plt.subplot(111)
#     sns.heatmap(np.sum(intensityMatrix, axis = 2), cmap = colormap, yticklabels = yTicks, xticklabels = xTicks)
# sns.heatmap(np.mean(sum_R3_matrix, axis = 2), cmap = 'Reds', yticklabels = False, xticklabels = False, vmin = vim, vmax = vmax)
sns.heatmap(np.mean(sum_r3_T0_matrix, axis = 0), cmap = 'Reds', yticklabels = yTicks, xticklabels = xTicks, vmin = 0, vmax = vmax_R3)
ax2.set_ylabel('Angle', fontsize=16)
ax2.set_xlabel('length from R-heel', fontsize=16)
ax2.set_title('Sum over z-stack', fontsize=16)
ax2.invert_yaxis()

In [ ]:
"""Get min/max params"""
sliceType = 0
sum_GFP_matrix = np.sum(gfp_fz1_type2, axis = 3)
sum_r3_R3_matrix = np.sum(r3_fz1_R3_type2, axis = 3)
sum_r3_R4_matrix = np.sum(r3_fz1_R4_type2, axis = 3)
sum_r3_T0_matrix = np.sum(r3_fz1_T0_type2, axis = 3)

# vmax_GFP = np.ceil(np.percentile(sum_GFP_matrix.flatten(), 99))
# vmax_R3 = np.ceil(np.percentile(sum_r3_R3_matrix.flatten(), 95))



vmax_GFP, vmax_R3

# tickParams = [1, 2, 5, [-1.5, -1, -0.5, 0, 0.5, 1, 1.5]] ### tickTypeX, tickTypeY, tickArg2_X, tickArg2_Y
tickTypeX = 1
tickTypeY = 2
tickArg2_X = 5
tickArg2_Y = [-1.5, -1, -0.5, 0, 0.5, 1, 1.5]

pp = outputDatas[0]['Parameter'][0][0][1]
xTick_ori = np.round(np.linspace(0, radiusExpanseRatio, sum_GFP_matrix[0,:,:].shape[1]), 2)
xTicks = my.getTickList(tickTypeX, xTick_ori, tickArg2_X)
yTick_ori = np.round(pp[5], 2)
yTicks = my.getTickList(tickTypeY, yTick_ori, tickArg2_Y)

ax2 =  plt.subplot(111)
#     sns.heatmap(np.sum(intensityMatrix, axis = 2), cmap = colormap, yticklabels = yTicks, xticklabels = xTicks)
# sns.heatmap(np.mean(sum_GFP_matrix, axis = 2), cmap = 'Greens', yticklabels = False, xticklabels = False, vmin = vim, vmax = vmax)
sns.heatmap(np.mean(sum_GFP_matrix, axis = 0), cmap = 'Greens', yticklabels = yTicks, xticklabels = xTicks, vmin = 0, vmax = vmax_GFP)
ax2.set_ylabel('Angle', fontsize=16)
ax2.set_xlabel('length from R-heel', fontsize=16)
ax2.set_title('Sum over z-stack', fontsize=16)
ax2.invert_yaxis()

In [ ]:
ax2 =  plt.subplot(111)
#     sns.heatmap(np.sum(intensityMatrix, axis = 2), cmap = colormap, yticklabels = yTicks, xticklabels = xTicks)
# sns.heatmap(np.mean(sum_R3_matrix, axis = 2), cmap = 'Reds', yticklabels = False, xticklabels = False, vmin = vim, vmax = vmax)
sns.heatmap(np.mean(sum_r3_T0_matrix, axis = 0), cmap = 'Reds', yticklabels = yTicks, xticklabels = xTicks, vmin = 0, vmax = vmax_R3)
ax2.set_ylabel('Angle', fontsize=16)
ax2.set_xlabel('length from R-heel', fontsize=16)
ax2.set_title('Sum over z-stack', fontsize=16)
ax2.invert_yaxis()